In [35]:
# Remove all mentions of blouse_DF and sunderas before writing to CSV, directly after making dataframe
# Another approach is to remove the JSON files of everything not MORIA.... and the MIROA with blouse_DF etc.
import json
import pandas as pd
from pandas.io.json import json_normalize 
from pandasql import sqldf
import os
from os import listdir
from os.path import isfile, join
DATAPATH = "../../data/Datasets_Enhancy/Annotation/"
IMAGEPATH = "../../data/Datasets_Enhancy/datasets/"

onlyfiles = [f for f in listdir(DATAPATH) if isfile(join(DATAPATH, f))]
ret = []
for f in onlyfiles:
    if ("MIROA") in f and not ("blouse" in f or "sudaderas" in f or "top_DF" in f):
        ret.append(f)
print(ret)
frames =[]
for i in range(len(ret)):
    with open(DATAPATH + ret[i]) as f:
        d = json.load(f)
        frames.append(pd.json_normalize(d['annotations']))
        
df = pd.concat(frames)
cols = [x for x in df.columns if "attributes" in x]
print(cols)
df["attributes"] = "-1"
df["attributes"] = df[cols].T.tolist()
exportDf = df[["image_id", "attributes"]]
exportDf

['MIROA_Hauts_Anorak_Bomber_DF_Complet.json', 'MIROA_Full_body_petos_complet.json', 'MIROA_Full_body_monos_complet.json', 'MIROA_Bas_jeans_complet_retravailJBN.json', 'MIROA_Hauts_Blazer_DF_Complet.json', 'MIROA_Full_body_vestidos_complet.json', 'MIROA_Hauts_cardigans_complet.json', 'MIROA_Bas_faldas_complet.json', 'MIROA_Hauts_camisas_y_blusas_complet.json', 'MIROA_Hauts_jerseys_complet.json', 'MIROA_Full_body_jacket_DF_complet.json', 'MIROA_Full_Body_Romper_DF_Complet.json', 'MIROA_Full_body_abrigos_complet.json', 'MIROA_Full_body_coat_DF_complet.json', 'MIROA_Full_Body_jumpsuit_DF_Complet.json']
['attributes.Col', 'attributes.Motifs', 'attributes.Details', 'attributes.Manches', 'attributes.Matiere', 'attributes.Encolure', 'attributes.Longueur', 'attributes.Epaules_Denudees', 'attributes.Categorie_de_vetement', 'attributes.Coupe', 'attributes.Manches_Originales', 'attributes.Forme', 'attributes.Occasion', 'attributes.Taille_Hanches', 'attributes.Motifs_Imprimes_Taille', 'attributes.M

,image_id,attributes
0,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[Null, Uni, [Col_Fourrure_Ou_Imitation], Manch..."
1,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[Null, Uni, [Boutons, Zip_Et_Fermeture_Eclair_..."
2,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[Null, Uni, [Boutons, Poches], Manches_Longues..."
3,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[Null, Uni, [Zip_Et_Fermeture_Eclair_Stylistiq..."
4,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[Col_Classique, Serigraphie_Et_Illustration, [..."
...,...,...
3561,jumpsuit_DF/Woven_Ruffle_Cami_Jumpsuit/img_000...,"[Null, Uni, [Null], Bretelles, null, Encolure_..."
3562,jumpsuit_DF/Zippered_Colorblock_Crepe_Jumpsuit...,"[Null, Uni, [Null], Halter, null, Col_Chemine_..."
3563,jumpsuit_DF/Zippered_Colorblock_Crepe_Jumpsuit...,"[Null, Bi_Ou_Tri_Color, [Null], Sans_Manches, ..."
3564,jumpsuit_DF/Zippered_Colorblock_Crepe_Jumpsuit...,"[Null, Uni, [Null], Bretelles, null, Encolure_..."


In [33]:
deleted = 0
changed = 0
s = len(exportDf.index)
outPut = exportDf.copy()

for i in range(s):
    id = exportDf.image_id.iloc[i]
    path1 = IMAGEPATH + id
    path2 = IMAGEPATH + "dataset/" + id
    relPath1 =  id
    relPath2 = "dataset/" + id
    
    if not os.path.isfile(path1):
        if not os.path.isfile(path2):
            try:
                idx = outPut.loc[outPut['image_id'] == id].index[0]
                print("DROPPED AT idx {} path {}".format(i, path2))
                outPut = outPut.drop(idx, axis = 0)
                deleted += 1
            except:
                print("ALREADY DROPPED {}".format(path2))
            
newLen = len(outPut.index)
for i in range(newLen):
    id = outPut.image_id.iloc[i]
    if os.path.isfile(IMAGEPATH + "dataset/" + id):
        outPut.loc[outPut['image_id'] == id, "image_id"] = "dataset/" + id



print("Total: {}".format(s))    
print("Deleted: {}".format(deleted))
print("Edited: {}".format(changed))
#outPut.to_csv(r'./export_data.csv', index = False)
outPut

DROPPED AT idx 6816 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Boxy_Buttoned_Blazer/img_00000052.jpg
DROPPED AT idx 6817 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Boxy_Buttoned_Blazer/img_00000052.jpg
DROPPED AT idx 6826 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Boxy_Buttoned_Blazer/img_00000071.jpg
DROPPED AT idx 6849 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Button-Front_Blazer/img_00000004.jpg
DROPPED AT idx 6919 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Button-Front_Blazer/img_00000102.jpg
DROPPED AT idx 7036 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Classic_Boyfriend_Blazer/img_00000112.jpg
DROPPED AT idx 7086 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Classic_Cuffed_Blazer/img_00000043.jpg
DROPPED AT idx 7087 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Classic_Cuffed_Blazer/img_00000043.jpg
DROPPED AT idx 7090 path ../../data/Datasets_Enhancy

DROPPED AT idx 11535 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Tuxedo-Inspired_Blazer/img_00000083.jpg
DROPPED AT idx 11740 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Two-Button_Blazer/img_00000105.jpg
DROPPED AT idx 12005 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Zippered_Shawl_Collar_Blazer/img_00000052.jpg
DROPPED AT idx 12006 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Zippered_Shawl_Collar_Blazer/img_00000052.jpg
DROPPED AT idx 12052 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Zippered_Shawl_Collar_Blazer/img_00000107.jpg
DROPPED AT idx 12110 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Zippered_Single-Button_Blazer/img_00000032.jpg
DROPPED AT idx 12111 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Zippered_Single-Button_Blazer/img_00000032.jpg
DROPPED AT idx 12444 path ../../data/Datasets_Enhancy/datasets/dataset/blazer_DF/Watercolor_Floral_Print_Blazer/img_00000032.j

,image_id,attributes
0,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[Null, Uni, [Col_Fourrure_Ou_Imitation], Manch..."
1,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[Null, Uni, [Boutons, Zip_Et_Fermeture_Eclair_..."
2,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[Null, Uni, [Boutons, Poches], Manches_Longues..."
3,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[Null, Uni, [Zip_Et_Fermeture_Eclair_Stylistiq..."
4,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[Col_Classique, Serigraphie_Et_Illustration, [..."
...,...,...
3561,jumpsuit_DF/Woven_Ruffle_Cami_Jumpsuit/img_000...,"[Null, Uni, [Null], Bretelles, null, Encolure_..."
3562,jumpsuit_DF/Zippered_Colorblock_Crepe_Jumpsuit...,"[Null, Uni, [Null], Halter, null, Col_Chemine_..."
3563,jumpsuit_DF/Zippered_Colorblock_Crepe_Jumpsuit...,"[Null, Bi_Ou_Tri_Color, [Null], Sans_Manches, ..."
3564,jumpsuit_DF/Zippered_Colorblock_Crepe_Jumpsuit...,"[Null, Uni, [Null], Bretelles, null, Encolure_..."


In [34]:
outPut.to_csv(r'./export_labels_working.csv', index = False)


54451    NaN
Name: segmentation, dtype: object

,image_id,segmentation
353,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[[227, 548, 505, 548, 505, 977, 227, 977]]"
354,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[[23, 109, 346, 109, 346, 544, 23, 544]]"
355,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[[64, 78, 289, 78, 289, 380, 64, 380]]"
356,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[[46, 81, 268, 81, 268, 466, 46, 466]]"
357,anorak_bomber_DF/Hooded_Cotton_Canvas_Anorak/i...,"[[29, 124, 356, 124, 356, 555, 29, 555]]"
358,anorak_bomber_DF/Quilted_Faux_Leather_Bomber/i...,"[[57, 172, 557, 172, 557, 838, 57, 838]]"
359,anorak_bomber_DF/Quilted_Faux_Leather_Bomber/i...,"[[73, 54, 229, 54, 229, 269, 73, 269]]"
360,anorak_bomber_DF/Zip_Collar_Bomber/img_0000002...,"[[58, 113, 325, 113, 325, 409, 58, 409]]"
361,anorak_bomber_DF/Zip_Collar_Bomber/img_0000001...,"[[131, 278, 1105, 278, 1105, 1343, 131, 1343]]"
362,anorak_bomber_DF/Zip_Collar_Bomber/img_0000003...,"[[38, 50, 289, 50, 289, 381, 38, 381]]"
